## Preface.py

In [3]:
###############
### Imports ###
###############

import sys, os, shutil
from sys import getsizeof as gso
import psutil
import time
import glob
import random
import string
import yaml
import gc
import argparse
import pathlib
from itertools import chain, repeat, zip_longest
from memory_profiler import profile
from icecream import ic
import traceback
import math
from pynverse import inversefunc

# arrays and data packages
import numpy as np
import pandas as pd
import re
import h5py
from funcy import chunks

# astrophysics
from astropy import units as unit
from astropy import constants as const
import natpy as nat
import healpy as hp

# speed improvement
import numba as nb
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import Pool

# scipy packages
from scipy.integrate import solve_ivp, quad, simpson
from scipy.interpolate import UnivariateSpline, interp1d
from scipy.special import expit, zeta
import scipy.stats as stats

# plotting
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
from mpl_toolkits import mplot3d
import colorcet as cc
# import imageio

import code
def raise_sys_exit():
    raise SystemExit
DONE = {"done": raise_sys_exit}

def debug(local_vars):
    try:
        code.interact(local=local_vars)
    except SystemExit:
        sys.exit()

# note: 1 line of code for debugging
# debug(DONE | locals())  # use done() to exit interactive window


#############
### Plots ###
#############

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('figure', figsize=(8, 8))
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


#############
### Units ###
#############

GB_UNIT = 1000*1024**2
Pi = np.pi
hc_val = (const.h/(2*Pi)*const.c).convert(unit.eV*unit.cm).value

eV     = 1.                         # Unit of energy: eV
meV    = 1.0e-3*eV
keV    = 1.0e3*eV
MeV    = 1.0e3*keV
GeV    = 1.0e3*MeV
TeV    = 1.0e3*GeV
erg    = TeV/1.602                  # erg
J      = 1.0e7*erg                  # joule
K      = 8.61732814974493e-5*eV     # Kelvin

cm     = (1/hc_val)/eV              # centi-meter
m      = 1.0e2*cm
km     = 1.0e3*m
pc     = 3.08567758128e18*cm        # parsec
kpc    = 1.0e3*pc
Mpc    = 1.0e3*kpc

# note: 
# Natural units defined via c=1, i.e. s/m = 299792458
s      = 2.99792458e10*cm           # second
yr     = 365*24*60*60*s
Gyr    = 1e9*yr
t0     = 13.787*Gyr
Hz     = 1.0/s

kg     = J/m**2*s**2
gram   = kg/1000.
Msun   = 1.98847e30*kg              # Mass of the Sun
G      = 6.6743e-11*m**3/kg/s**2    # Gravitational constant
Da     = 1.66053906660e-27*kg       # Dalton or atomic mass unit (u)

deg    = Pi/180.0                   # Degree
arcmin = deg/60.                    # Arcminute
arcsec = arcmin/60.                 # Arcsecond
sr     = 1.                         # Steradian


##########################################
### Parameters - Mertsch et al. (2020) ###
##########################################

# NFW parameters for Milky Way.
Mvir_MW  = 2.03e12*Msun                            # Virial mass
rho0_MW  = 1.06e7*(Msun/kpc**3)                    # density normalization
Rs_MW    = 19.9*kpc                                # scale radius 
Rvir_MW  = 333.5*kpc                               # virial radius

# NFW parameters for Virgo Cluster.
Mvir_VC  = 6.9e14*Msun                             # Virial mass
rho0_VC  = 8.08e5*(Msun/kpc**3)                    # density normalization
Rs_VC    = 399.1*kpc                               # scale radius 
Rvir_VC  = 2328.8*kpc                              # virial radius

# Coordinates.
GLAT_VC = 74.44                                    # Galactic latitude
GLON_VC = 283.81                                   # Galactic longitude
DIST_VC = 16.5e3*kpc                               # Distance

# Translated to cartesian coordinates [kpc] in our setup (from fct.halo_pos).
X_VC     = np.array([-4289.63477282, 1056.51861602, 15895.27621304])

# NFW parameters for Andromeda.
Mvir_AG  = 8.0e11*Msun                             # Virial mass
rho0_AG  = 3.89e6*(Msun/kpc**3)                    # density normalization
Rs_AG    = 21.8*kpc                                # scale radius 
Rvir_AG  = 244.7*kpc                               # virial radius

# Coordinates.
GLAT_AG = -21.573311                               # Galactic latitude
GLON_AG = 121.174322                               # Galactic longitude
DIST_AG = 0.784e3*kpc                              # Distance

# Translated to cartesian coordinates [kpc] in our setup (from fct.halo_pos).
X_AG    = np.array([632.29742673, -377.40315121, -288.27006757])

#? for now temps. here
T_CMB = 2.72548*K
T_CNB = np.power(4/11, 1/3)*T_CMB

## Make box parameters.

In [40]:
# from shared.preface import *

def make_box_parameters(box_dir, box_name, box_ver, zi_snap, zf_snap):
    """
    Reads cosmological and other global parameters of the specified simulation box. Output is stored in corresponding output folder, where all simulation outputs (e.g. neutrino overdensities) will be stored.
    """

    # Box input/output paths.
    file_dir = f'{box_dir}/{box_name}/{box_ver}'
    out_dir = f'{os.getcwd()}/{box_name}/{box_ver}'

    # Create output directory. If it exists already, delete then create.
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
    os.makedirs(out_dir)

    # The snapshots to read, determined by inputs zi_snap and zf_snap.
    snap_nums = np.arange(zf_snap, zi_snap+1)
    nums_4cif = []
    zeds = np.zeros(len(snap_nums))

    # Loop over selected snapshots and save outputs.
    for i, num in enumerate(snap_nums):
        num_4cif = f'{num:04d}'
        nums_4cif.append(num_4cif)

        with h5py.File(f'{file_dir}/snapshot_{num_4cif}.hdf5') as snap:

            # Store redshifts.
            zeds[i] = snap['Cosmology'].attrs['Redshift'][0]

            # Save global parameters only once.
            if i == 0:

                # Cosmology.
                cosmo = snap['Cosmology']
                Omega_R = cosmo.attrs['Omega_r'][0]
                Omega_M = cosmo.attrs['Omega_m'][0]
                Omega_L = cosmo.attrs['Omega_lambda'][0]
                h_dimless = cosmo.attrs['h'][0]

                # Dark matter "particle" mass of box.
                DM_mass = np.unique(snap['PartType1/Masses'][:]*1e10*Msun)[0]

                # Gravity smoothening length of box.
                smooth_len = snap['GravityScheme'].attrs[
                    'Maximal physical DM softening length (Plummer equivalent) [internal units]'
                ][0]*1e6*pc



                # Create .yaml file for global parameters of box.
                box_parameters = {
                    "File Paths": {
                        "Box Directory": box_dir,
                        "Box Name": box_name,
                        "Box Version": box_ver,
                        "Box File Directory": file_dir,
                    },
                    "Cosmology": {
                        "Omega_R": float(Omega_R),
                        "Omega_M": float(Omega_M),
                        "Omega_L": float(Omega_L),
                        "h": float(h_dimless),
                    },
                    "Content": {
                        "DM Mass [Msun]": float(DM_mass/Msun),
                        "Smoothening Length [pc]": float(smooth_len/pc),
                    } 
                }
                with open(f'{out_dir}/box_parameters.yaml', 'w') as file:
                    yaml.dump(box_parameters, file)

    # Save 4cifer codes and redshift for snapshots, as array files.
    np.save(f'{out_dir}/zeds_snaps.npy', zeds)
    np.save(f'{out_dir}/nums_snaps.npy', nums_4cif)


make_box_parameters(
    box_dir='/projects/0/einf180/Tango_sims', 
    box_name='L025N752', 
    box_ver='DMONLY/SigmaConstant00', 
    zi_snap=36, 
    zf_snap=13
)

## Make simulation parameters.

In [41]:
# from shared.preface import *

def make_sim_parameters(
        sim_dir, sim_type,
        nu_mass_start, nu_mass_stop, nu_mass_num, nu_sim_mass,
        p_start, p_stop, p_num,
        phis, thetas,
        init_x_dis,
        z_int_shift, z_int_stop, z_int_num, 
        int_solver,
        CPUs_precalculations, CPUs_simulations, memory_limit_GB,
        DM_in_cell_limit, 
    ):

    # Load simulation box parameters.
    with open(f'{sim_dir}/box_parameters.yaml', 'r') as file:
        box_params = yaml.safe_load(file)

    h = box_params['Cosmology']['h']
    H0 = h*100*km/s/Mpc
    Omega_M = box_params['Cosmology']['Omega_M']
    Omega_L = 1.-Omega_M  # since we don't use Omega_R


    ### ============================== ###
    ### Simulation control parameters. ###
    ### ============================== ###

    def s_of_z(z):
        """
        Convert redshift to time variable s with eqn. 4.1 in Mertsch et al.
        (2020), keeping only Omega_M and Omega_L in the Hubble eqn. for H(z).

        Args:
            z (float): redshift

        Returns:
            float: time variable s (in [seconds] if 1/H0 factor is included)
        """    

        def s_integrand(z):        

            # We need value of H0 in units of 1/s.
            H0_val = H0/(1/s)
            a_dot = np.sqrt(Omega_M*(1.+z)**3 + Omega_L)/(1.+z)*H0_val
            s_int = 1./a_dot

            return s_int

        s_of_z, _ = quad(s_integrand, 0., z)

        return np.float64(s_of_z)

    # Number of simulated neutrinos.
    if isinstance(phis, int):
        neutrinos = phis*thetas*p_num
    else:
        neutrinos = len(phis)*len(thetas)*p_num

    # Neutrino masses.
    nu_mass_eV = nu_sim_mass*eV
    nu_mass_kg = nu_mass_eV/kg
    neutrino_massrange_eV = np.geomspace(
        nu_mass_start, nu_mass_stop, nu_mass_num
    )*eV

    # Neutrino momentum range.
    neutrino_momenta = np.geomspace(p_start, p_stop, p_num)

    # Neutrino + antineutrino number density of 1 flavor in [1/cm**3],
    # using the analytical expression for Fermions.
    n0 = 2*zeta(3.)/Pi**2 *T_CNB**3 *(3./4.) /(1/cm**3)

    # Starting position of neutrinos.
    neutrino_init_position = np.array([init_x_dis, 0., 0.])

    # Logarithmic redshift spacing, and conversion to integration variable s.
    z_int_steps = np.geomspace(z_int_shift, z_int_stop+z_int_shift, z_int_num)
    z_int_steps -= z_int_shift
    s_int_steps = np.array([s_of_z(z) for z in z_int_steps])

    #? these two figure out later.
    # For sphere of incluence tests: Divide inclusion region into shells.
    DM_shell_edges = np.array([0,5,10,15,20,40,100])*100*kpc
    # Multiplier for DM limit for each shell.
    shell_multipliers = np.array([1,3,6,9,12,15])


    ### ============================================ ###
    ### Create .yaml file for simulation parameters. ###
    ### ============================================ ###

    sim_parameters = {
        "neutrinos": neutrinos,
        "initial_haloGC_distance": init_x_dis,
        "neutrino_mass_start": nu_mass_start,
        "neutrino_mass_stop": nu_mass_stop,
        "neutrino_mass_num": nu_mass_num,
        "neutrino_simulation_mass_eV": nu_mass_eV,
        "neutrino_simulation_mass_kg": float(nu_mass_kg),
        "phis": phis,
        "thetas": thetas,
        "momentum_start": p_start,
        "momentum_stop": p_stop,
        "momentum_num": p_num,
        "z_inegration_start": 0,
        "z_inegration_stop": z_int_stop,
        "z_inegration_num": z_int_num,
        "integration_solver": int_solver,
        "CPUs_precalculations": CPUs_precalculations,
        "CPUs_simulations": CPUs_simulations,
        "memory_limit_GB": memory_limit_GB,
        "DM_in_cell_limit": DM_in_cell_limit,
        "cosmo_neutrino_density": float(n0)
    }

    with open(f'{sim_dir}/sim_parameters.yaml', 'w') as file:
        yaml.dump(sim_parameters, file)

    # Save arrays as .npy files.
    np.save(f'{sim_dir}/neutrino_massrange_eV.npy', neutrino_massrange_eV)
    np.save(f'{sim_dir}/neutrino_momenta.npy', neutrino_momenta)
    np.save(f'{sim_dir}/neutrino_init_position.npy', neutrino_init_position)
    np.save(f'{sim_dir}/z_int_steps.npy', z_int_steps)
    np.save(f'{sim_dir}/s_int_steps.npy', s_int_steps)
    np.save(f'{sim_dir}/DM_shell_edges.npy', DM_shell_edges)
    np.save(f'{sim_dir}/shell_multipliers.npy', shell_multipliers)


    ### ================================ ###
    ### Initial velocities of neutrinos. ###
    ### ================================ ###
    
    # Convert momenta to initial velocity magnitudes, in units of [kpc/s].
    u_i = neutrino_momenta/nu_mass_eV / (kpc/s)

    if sim_type == 'all_sky':
        # For all-sky script, input is just one coord. pair.
        t, p = thetas, phis

        # Each coord. pair gets whole momentum, i.e. velocity range.
        uxs = [-u*np.cos(p)*np.sin(t) for u in u_i]
        uys = [-u*np.sin(p)*np.sin(t) for u in u_i]
        uzs = [-u*np.cos(t) for u in u_i]

        u_i_array = np.array(
            [[ux, uy, uz] for ux,uy,uz in zip(uxs,uys,uzs)]
        )

    else:
        # Split up this magnitude into velocity components, by using spher. 
        # coords. trafos, which act as "weights" for each direction.
        cts = np.linspace(-1, 1, thetas)  # cos(thetas)
        ps = np.linspace(0, 2*Pi, phis)   # normal phi angles

        # Minus signs due to choice of coord. system setup (see notes/drawings).
        #                              (<-- outer loops, --> inner loops)
        uxs = [
            -u*np.cos(p)*np.sqrt(1-ct**2) for ct in cts for p in ps for u in u_i
        ]
        uys = [
            -u*np.sin(p)*np.sqrt(1-ct**2) for ct in cts for p in ps for u in u_i
        ]
        uzs = [
            -u*ct for ct in cts for _ in ps for u in u_i
        ]

        u_i_array = np.array(
            [[ux, uy, uz] for ux,uy,uz in zip(uxs,uys,uzs)]
        )

    np.save(f'{sim_dir}/initial_velocities.npy', u_i_array)


make_sim_parameters(
    sim_dir='L025N752/DMONLY/SigmaConstant00', 
    sim_type='halo_batch',
    nu_mass_start=0.01, 
    nu_mass_stop=0.3, 
    nu_mass_num=100, 
    nu_sim_mass=0.3,
    p_start=0.01, 
    p_stop=400, 
    p_num=100,
    phis=10, 
    thetas=10,
    init_x_dis=8.5,
    z_int_shift=1e-1, 
    z_int_stop=4, 
    z_int_num=100, 
    int_solver='RK23',
    CPUs_precalculations=128, 
    CPUs_simulations=128, 
    memory_limit_GB=224,
    DM_in_cell_limit=10_000, 
)

## Initializing simulation setup

In [ ]:
from shared.preface import *
import shared.functions as fct



## Figures.

In [ ]:
# GOAL:
def read_overdensities(box_name, box_ver, sim_fullname):
    # sim_fullname would then be super dynamical: e.g. 'halo_batch_40k_test1' 
    # or 'spheres_5shells_10k_final'

    # don't care about number of halos in batch: just read all available 
    # overdensities, and display number of halos that were used -> i.e. halo 
    # batch size not as input.

    # The output would be in dynaimcal named folder, which get created after 
    # the sims.
    # All overdensities of that sim type would then be in folder: 
    # f'sim_outputs/{box_name}/{box_ver}/{sim_fullname}/'

    # Also in that folder, will be a .yaml file with the box parameters.

In [ ]:
def read_overdensities_halo_batch(
        mass_gauge, mass_range, size 
    ):

    hname = f'1e+{mass_gauge}_pm{mass_range}Msun'
    fct.halo_batch_indices(
        PRE_BOX.Z0_STR, mass_gauge, mass_range, 'halos', size, 
        hname, PRE_BOX.SIM_DIR, PRE_BOX.OUT_DIR
    )
    halo_batch_params = np.load(f'{PRE_BOX.OUT_DIR}/halo_batch_{hname}_params.npy')
    halo_num = len(halo_batch_params)

    print('********Number density band********')
    print('Halo batch params (Rvir,Mvir,cNFW):')
    print(halo_batch_params)
    print('***********************************')


    etas_arr = []
    for halo_j in range(halo_num):

        # Load number densities.
        tot_name = f'{hname}_halo{halo_j}'
        fname = f'{PRE_BOX.NUS}nus_{tot_name}'
        out_file = f'{PRE_BOX.OUT_DIR}/number_densities_band_poss_{fname}.npy'
        etas = np.load(f'{out_file}')/N0
        etas_arr.append(etas)

    etas_sim = np.array(etas_arr).reshape(halo_num, len(NU_MRANGE))

## Creating .yaml file.

In [39]:
from shared.preface import *

import yaml

user_input_yaml = {
    "Cosmology": {
        "h": h,
        "Omega_R": Omega_R,
        "Omega_M": Omega_M,

    } 
}
with open('test.yaml', 'w') as file:
    yaml.dump(user_input_yaml, file)


with open('test.yaml', 'r') as file:
    setup = yaml.safe_load(file)


setup['Cosmology']['h']

NameError: name 'Omega_R' is not defined